In [1]:
import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [2]:
# Avoid hogging up gpu memory 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPUs


In [3]:
# For convenience
%load_ext autoreload
%autoreload 2

In [4]:
from tensorflow import keras
import json
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import dataset
from util import (
    get_place_to_index_mapping,
    get_incident_to_index_mapping
)
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Flatten, Permute
from keras import Sequential
import keras.backend as kb
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import PIL

In [5]:
abs_path = "/kuacc/users/asafaya19/cv-project"
train_json = os.path.join(abs_path ,"eccv_train.json")
val_json = os.path.join(abs_path ,"eccv_val.json")
data_dir = os.path.join(abs_path, "data")
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")

train_paths = json.loads(open(train_json).readline())
val_paths = json.loads(open(val_json).readline())

place_to_idx = get_place_to_index_mapping()
incident_to_idx = get_incident_to_index_mapping()

In [6]:
def get_dataset(paths,file_dir, threshold=1000):
    train_set = []
    for path in tqdm(paths, leave=False):
    #     print(path)
        if not os.path.exists(os.path.join(file_dir, path)):
            continue
        # Make sure image is not corrupt, try importing it
        try:
            img = PIL.Image.open(os.path.join(file_dir, path))
            img.resize((224, 224))
        except:
            continue
        nump = len(place_to_idx) + 1
        numi = len(incident_to_idx) + 1
        place_labels = np.zeros(nump, np.float32)
        place_weights = np.zeros(nump, np.float32)
        incident_labels = np.zeros(numi, np.float32)
        incident_weights = np.zeros(numi, np.float32)

        incidents = paths[path]["incidents"]
        for k in incidents:
            lbl = incidents[k]
            if lbl==1:
                # We are sure this instance is only this incident
                incident_labels[incident_to_idx[k]]=1
                incident_weights = np.ones(numi, np.float32)
            else:
                # We are only sure that this image is not that incident
                incident_weights[incident_to_idx[k]]=1
        if len(incidents)==0:
            # No incident
            incident_labels[-1]=1
            incident_weights = np.ones(numi, np.float32)

        places = paths[path]["places"]
        for k in places:
            lbl = places[k]
            if lbl==1:
                # We are sure this instance is only this incident
                place_labels[place_to_idx[k]]=1
                place_weights = np.ones(nump, np.float32)
            else:
                # We are only sure that this image is not that incident
                place_weights[place_to_idx[k]]=1
        if len(places)==0:
            # No place
            place_labels[-1]=1
            place_weights = np.ones(nump, np.float32)


        train_set.append({
            "path":path,
            "incident_labels":incident_labels,
            "incident_weights":incident_weights,
            "incidents":np.vstack((incident_labels, incident_weights)),
            "place_labels":place_labels,
            "place_weights":place_weights,
            "place":np.vstack((place_labels, place_weights))
        })
        if len(train_set)>=threshold:
            break
    return train_set

In [7]:
def getpreprocessfunc():
    mean = np.asarray([0.485, 0.456, 0.406]).reshape(3, 1, 1).astype(np.float32)
    std = np.asarray([0.229, 0.224, 0.225]).reshape(3, 1, 1).astype(np.float32)
    def preprocessfunc(img):
#         print(img.shape)
#         print(type(img))
        img /= 255
        img -= mean
        img /= std
        return img
    return preprocessfunc

In [8]:
# Enclosure to retain state
def get_weighted_accuracy():
    m = keras.metrics.CategoricalAccuracy()
    def weighted_accuracy(y_true, y_preds):
        y_true = tf.reshape(y_true, (bs, 2, -1))
        y_true_lbls = y_true[:,0,:]
        return m(y_true_lbls, y_preds)
    return weighted_accuracy

In [9]:
def weighted_loss(y_true, y_preds):
    bce = keras.losses.BinaryCrossentropy(keras.losses.Reduction.NONE)
#     tf.print(y_true)
#     tf.print(y_preds)
#     print(y_true.shape)
#     print(y_preds.shape)
    bs = y_true.shape[0]
    
    print(y_true.shape, y_preds.shape)
    
    y_true = tf.reshape(y_true, (bs, 2, -1))
    y_true_lbls = y_true[:,0,:]
    y_true_weights = y_true[:,1,:]
    bce_loss = bce(y_true_lbls, y_preds)
#     import pdb
#     pdb.set_trace()
#     return bce_loss
    return tf.reduce_sum(tf.multiply(bce_loss, y_true_weights))

In [10]:
# from keras.models import model_from_json


# class TrunkModel(keras.Model):
#     def __init__(self, trunk_model, incident_weights, place_weights):
#         super(FinalModel, self).__init__()
#         self.trunk_model = trunk_model
#         self.incident_proj = Dense(len(incident_to_idx), input_shape=(1024,), name="incidents_projection", weights=incident_weights)
#         self.places_proj = Dense(len(place_to_idx), input_shape=(1024,), name="places_projection", weights=place_weights)
        
#     def call(self, inputs):
# #         x = self.permute(inputs)
#         x = self.cropped(inputs)
#         x = self.permuteback(x)
#         x = self.trunk_model(x)
        
#         return self.incident_proj(x), self.places_proj(x)

In [23]:
from keras.models import model_from_json


class FinalModel(keras.Model):
    def __init__(self, trunk_model, incident_weights, place_weights):
        super(FinalModel, self).__init__()
        self.permute = Permute((2, 3, 1))
        self.cropped = keras.layers.experimental.preprocessing.RandomCrop(224, 224)
        self.permuteback = Permute((3, 1, 2))
        self.trunk_model = trunk_model
        self.incident_proj = Dense(len(incident_to_idx), input_shape=(1024,), name="incidents_projection", weights=incident_weights)
        self.places_proj = Dense(len(place_to_idx), input_shape=(1024,), name="places_projection", weights=place_weights)
        
    def call(self, inputs):
        x = self.permute(inputs)
        x = self.cropped(x)
        x = self.permuteback(x)
        x = self.trunk_model(x)
        
        return self.incident_proj(x), self.places_proj(x)

In [12]:
# train_set = get_dataset(train_paths, train_dir, 1000)
train_set = get_dataset(train_paths, train_dir, 10)
val_set = get_dataset(val_paths, val_dir, 10)

train_df = pd.DataFrame(train_set)
val_df = pd.DataFrame(val_set)

In [13]:
imgen = ImageDataGenerator(
    horizontal_flip=True,
    preprocessing_function=getpreprocessfunc(),
)

imgen = imgen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col="path",
    y_col=["incidents", "place"],
    weight_col=None,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode="multi_output",
    batch_size=64,
    shuffle=False,
    seed=True,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset=None,
    interpolation="nearest",
    validate_filenames=True,
)

Found 10 validated image filenames.


In [14]:
losses = {
    "output_1": weighted_loss,
    "output_2": weighted_loss,
}

In [15]:
tf.__version__

'2.4.0'

In [16]:
import resnet

In [17]:
import torch

In [18]:
# ?keras.layers.experimental.preprocessing.RandomCrop

In [19]:
# keras.layers.experimental.preprocessing.RandomCrop((3, 224, 224))(np.random.randn(1,3, 256, 256))

TypeError: __init__() missing 1 required positional argument: 'width'

In [20]:
inp = keras.Input(shape=(3, 256, 256))
cropped = keras.layers.experimental.preprocessing.RandomCrop(3, 224, 224)(inp)
# model = ResNet50(include_top=False, pooling = "avg", weights="imagenet")(cropped)
# trunk = Dense(1024, activation="relu", name="trunk")(model)
trunk = resnet.trunk()
resnet.init_weights_from_torch(trunk, "/kuacc/users/shamdan17/cv/IncidentsDataset/pretrained_weights/eccv_final_model_trunk.pth.tar")

In [24]:
opt = keras.optimizers.Adam(lr=1e-5)

# trunk_model = model_from_json(open(os.path.join(abs_path,'models/trunk.json'), 'r').read())
# trunk_model.load_weights(os.path.join(abs_path,"models/trunk.h5"))
place_w = np.load(os.path.join(abs_path,'models/place_w.npy')).T
place_b = np.load(os.path.join(abs_path,'models/place_b.npy')).T
incident_w = np.load(os.path.join(abs_path,'models/incident_w.npy')).T
incident_b = np.load(os.path.join(abs_path,'models/incident_b.npy')).T

mdl = FinalModel(trunk, [incident_w, incident_b], [place_w, place_b])

mdl.compile(optimizer=opt, loss=losses, metrics=[get_weighted_accuracy()])

In [25]:
x, y = next(imgen)
o = mdl(x, training=False)

In [26]:
o

(<tf.Tensor: shape=(10, 43), dtype=float32, numpy=
 array([[ -7.1617665 ,  -4.8341255 ,  -4.2903466 ,  -6.3418193 ,
          -9.380282  ,  -4.4043617 ,  -9.522434  ,  -3.4043517 ,
          -4.7348557 ,  -4.297809  ,  -5.3797116 ,  -6.4514937 ,
          -6.299391  ,  -4.9656057 ,  -4.863049  ,  -3.090029  ,
          -4.5712547 ,  -4.314076  ,  -5.7307844 ,  -9.096714  ,
          -7.843625  ,  -4.1009216 ,  -7.644072  ,  -3.781779  ,
          -3.6550143 ,  -8.522072  ,  -2.4068387 ,  -7.84911   ,
          -5.88979   ,  -4.6173177 ,  -4.505504  ,  -8.359604  ,
          -8.286058  ,  -8.392991  ,  -8.128646  ,  -9.057092  ,
          -7.056719  ,  -7.635836  ,  -8.337256  ,  -9.846811  ,
         -11.560063  ,  -3.783824  ,  -6.4505286 ],
        [ -9.09824   , -10.97041   ,  -8.702867  ,  -6.4431343 ,
         -10.122774  ,   1.0165751 , -10.852257  ,  -8.753836  ,
          -9.618648  ,  -9.080101  ,  -2.158973  ,  -9.106092  ,
         -12.510469  , -10.714614  , -12.404375  ,  

In [27]:
from functools import partial
import sklearn
from sklearn.metrics._base import type_of_target

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def fixed_precision_recall_curve(y_true, probas_pred, *, pos_label=None,
                           sample_weight=None):

    fps, tps, thresholds = sklearn.metrics._ranking._binary_clf_curve(y_true, probas_pred,
                                             pos_label=pos_label,
                                             sample_weight=sample_weight)

    precision = tps / (tps + fps)
    precision[np.isnan(precision)] = 0
    recall = np.ones(tps.size) if tps[-1] == 0 else tps / tps[-1]

    # stop when full recall attained
    # and reverse the outputs so recall is decreasing
    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)
    return np.r_[precision[sl], 1], np.r_[recall[sl], 0], thresholds[sl]


def average_precision_score(y_true, y_score, *, average="macro", pos_label=1,
                            sample_weight=None):
    def _binary_uninterpolated_average_precision(
            y_true, y_score, pos_label=1, sample_weight=None):
        precision, recall, _ = fixed_precision_recall_curve(
            y_true, y_score, pos_label=pos_label, sample_weight=sample_weight)
        # Return the step function integral
        # The following works because the last entry of precision is
        # guaranteed to be 1, as returned by precision_recall_curve
        return -np.sum(np.diff(recall) * np.array(precision)[:-1])

    y_type = type_of_target(y_true)
    if y_type == "multilabel-indicator" and pos_label != 1:
        raise ValueError("Parameter pos_label is fixed to 1 for "
                         "multilabel-indicator y_true. Do not set "
                         "pos_label or set pos_label to 1.")
    elif y_type == "binary":
        # Convert to Python primitive type to avoid NumPy type / Python str
        # comparison. See https://github.com/numpy/numpy/issues/6784
        present_labels = np.unique(y_true).tolist()
        if len(present_labels) == 2 and pos_label not in present_labels:
            raise ValueError(
                f"pos_label={pos_label} is not a valid label. It should be "
                f"one of {present_labels}"
            )
    average_precision = partial(_binary_uninterpolated_average_precision,
                                pos_label=pos_label)
    return sklearn.metrics._base._average_binary_score(average_precision, y_true, y_score,
                                 average, sample_weight=sample_weight)

In [30]:
val_set = get_dataset(val_paths, val_dir, threshold=2000)
val_df = pd.DataFrame(val_set)

imgen = ImageDataGenerator(
    preprocessing_function=getpreprocessfunc(),
)

imgen = imgen.flow_from_dataframe(
    val_df,
    directory=val_dir,
    x_col="path",
    y_col=["incidents", "place"],
    weight_col=None,
    target_size=(256, 256),
    color_mode="rgb",
    classes=None,
    class_mode="multi_output",
    batch_size=64,
    shuffle=True,
    seed=True,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    subset=None,
    interpolation="nearest",
    validate_filenames=True,
)

  3%|▎         | 1864/57207 [00:41<23:29, 39.28it/s]  /kuacc/users/shamdan17/.conda/envs/cv/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:792: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Found 2000 validated image filenames.


In [31]:
imgen.reset()

nbatches = 0
imap = 0
pmap = 0

for x, y in tqdm(imgen):
    
    y1 = y[0]
    y2 = y[1]

    out = mdl.predict(x)
    
    iout = sigmoid(out[0])
    pout = sigmoid(out[1])
    
    itrue = y1[:,0,:-1]
    ptrue = y2[:,0,:-1]
    
    imap += average_precision_score(itrue, iout)
    pmap += average_precision_score(ptrue, pout)
    nbatches += 1
    
    if nbatches > np.ceil(val_df.shape[0] / 64):
        break
    
pmap.item() / nbatches, imap.item() / nbatches

 56%|█████▋    | 18/32 [00:28<00:19,  1.43s/it]/kuacc/users/shamdan17/.conda/envs/cv/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:792: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))
100%|██████████| 32/32 [00:51<00:00,  1.61s/it]


(0.09458364506874781, 0.32597322197714357)

In [37]:
np.argmax(itrue, axis=1)

array([ 0,  2,  0,  0, 16, 19, 35,  0, 28,  0, 33,  8, 20, 19,  0,  0,  0,
        0,  1,  0, 10,  0,  0,  0,  0,  6, 26,  0,  0,  0,  0, 19,  0,  0,
        0,  4,  0,  0,  0,  0,  1,  0, 28,  0,  1,  0,  0, 25, 27,  0,  0,
        0,  0,  0, 27, 14,  0,  0,  0,  0,  0, 20,  0,  0])

In [38]:
np.argmax(iout, axis=1)

array([28,  2, 31, 12, 16, 19, 35, 31, 28, 30, 33,  8, 20,  9, 34, 31, 11,
       11,  1,  0, 10,  1,  5,  9,  5,  6, 26,  7,  9, 39, 37, 19, 28,  1,
       23,  4, 20, 13, 11, 12,  1,  7, 27, 24,  1, 28, 27, 25, 27, 20,  2,
       19, 26,  9, 27, 14,  5, 19,  0, 34,  6, 20,  5, 32])

In [50]:
np.sum(itrue, axis=1)

array([0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [51]:
itrue[-11,:]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [52]:
iout[-11, :]

array([7.8323577e-03, 1.6026037e-03, 1.7557660e-02, 7.3732279e-04,
       2.9460841e-03, 5.5602047e-04, 6.5967061e-02, 1.9446540e-01,
       2.5903197e-02, 4.8334238e-01, 1.1569914e-04, 1.9244132e-04,
       9.0257805e-03, 6.8710232e-04, 4.2367685e-03, 7.4228289e-04,
       5.1610247e-04, 7.5694692e-04, 3.0503513e-03, 2.4045019e-03,
       2.5017953e-05, 6.4356536e-05, 1.8595252e-06, 2.0883859e-05,
       8.2792780e-05, 1.1010447e-04, 5.8081484e-04, 3.6724824e-03,
       2.9840881e-05, 7.4788538e-04, 2.2584569e-02, 5.0150946e-04,
       4.2383707e-04, 5.4173637e-04, 1.8666338e-05, 4.3027550e-05,
       1.2629124e-05, 3.4202083e-05, 4.0687784e-05, 2.9055998e-05,
       6.0458813e-05, 3.7572230e-03, 1.6195729e-02], dtype=float32)

In [47]:
itrue.shape

(64, 43)